In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import pipeline
import torch
import numpy as np
import os 
from datetime import datetime
import textwrap
import random

2025-10-28 21:14:45.355833: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-28 21:14:45.433583: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-28 21:14:49.585065: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [28]:
RANDOM_STATE = 42
path_DATA = "../../../../data"

# Dataset path
csv_path = f"{path_DATA}/spotify_dataset_sin_duplicados_4.csv"
zero_shot_path = f"{path_DATA}/zero_outputs"

# Zero-shot file path
path_df_zero_test = f"{zero_shot_path}/zero_shot_test_results.csv"
path_df_zero_train = f"{zero_shot_path}/zero_shot_train_results.csv"

# Prompting path
prompting_path_test = f"{zero_shot_path}/llama_8B_results_test.csv"
prompting_path_train = f"{zero_shot_path}/llama_8B_results.csv"

TESTING = False

if TESTING:
    NROWS = 50
else:
    NROWS = None
    


In [41]:
def get_array(path):
    with open(path, "r") as f:
        array_ = json.load(f)  
    return array_

def get_song_and_target(csv_path, faltantes_path = "faltantes_according_token.json", sample_size=None, path_test_indexes=None):
    df = pd.read_csv(csv_path)
    indices_to_remove = get_array(faltantes_path)
    if indices_to_remove[-1]>df.shape[0]:
        print("You are executing in testing way")
    else:
        df = df.drop(indices_to_remove).reset_index(drop=True)

    df['original_index'] = df.index

    if path_test_indexes is not None:
        df_split_indices_test = pd.read_csv(path_test_indexes)
        if 'train_index' not in df_split_indices_test.columns:
            idx_test = df_split_indices_test["test_index"].values
        else:
            idx_test = df_split_indices_test["train_index"].values
        df = df.iloc[idx_test].reset_index(drop=True)

    if sample_size is not None:
        df = df.iloc[:sample_size].reset_index(drop=True)
    

    
    # X = df['text']
    df['Explicit_binary'] = (df['Explicit'].str.lower() == 'yes').astype(int)
    # y = df['Explicit_binary']
    return df

In [44]:
# Load dataset
df_song_train = get_song_and_target(csv_path,"faltantes_according_token.json", sample_size=NROWS, path_test_indexes=f"{zero_shot_path}/split_indices_train.csv")
df_song_test = get_song_and_target(csv_path,"faltantes_according_token.json", sample_size=NROWS, path_test_indexes=f"{zero_shot_path}/split_indices_test.csv")
df_original = get_song_and_target(csv_path,"faltantes_according_token.json", sample_size=NROWS, path_test_indexes=None)
display(df_song_train.head())
print("df_song_train shape:", df_song_train.shape)
display(df_song_test.head())
print("df_song_test shape:", df_song_test.shape)

display(df_original.head())
print("df_original shape:", df_original.shape)

,Artist(s),song,text,Length,emotion,Genre,Album,Release Date,Key,Tempo,...,Similar Artist 2,Similar Song 2,Similarity Score 2,Similar Artist 3,Similar Song 3,Similarity Score 3,song_normalized,artist_normalized,original_index,Explicit_binary
0,Five Finger Death Punch,Battle Born,"[Verse 1] Once upon a time, I swore I had a he...",03:43,sadness,"hard rock,heavy metal",The Wrong Side Of Heaven And The Righteous Sid...,18th November 2013,B Maj,148,...,The Amity Affliction,Dont Lean on Me,0.985262,Aviana,Obsession,0.981057,battle born,five finger death punch,31344,1
1,John Prine,Souvenirs,All the snow has turned to water Christmas day...,03:42,sadness,"folk,country",Souvenirs,31st October 2000,E Maj,153,...,Ed Sheeran,Da Cipher,0.982120,Kimya Dawson,The Competition,0.978866,souvenirs,john prine,45251,0
2,Brad Paisley,We Danced,[Verse 1] The bar was empty I was sweepin' up ...,03:46,sadness,country,Who Needs Pictures,1st June 1999,A Maj,146,...,Parker McCollum,Cobra,0.983223,Whitney Houston,Exhale Shoop Shoop,0.980294,we danced,brad paisley,13407,0
3,John Prine,Aint Nobody,Verse 1: You said you would never leave me No...,05:01,joy,"folk,country",Lost Dogs + Mixed Blessings,4th April 1995,C Maj,98,...,Shayne Ward,No U Hang Up,0.974879,Sheryl Crow,Leaving Las Vegas,0.973937,aint nobody,john prine,45259,0
4,Greentea Peng,Your Mind,"[Intro] Sorry to say, sorry to say But I'm her...",04:19,sadness,soul,GREENZONE 108,9th September 2022,F# Maj,136,...,Ace of Hearts,Silver Lining,0.984751,"Star Cast,Luke James",So Sick,0.975151,your mind,greentea peng,36024,0


df_song_train shape: (86500, 43)


,Artist(s),song,text,Length,emotion,Genre,Album,Release Date,Key,Tempo,...,Similar Artist 2,Similar Song 2,Similarity Score 2,Similar Artist 3,Similar Song 3,Similarity Score 3,song_normalized,artist_normalized,original_index,Explicit_binary
0,Scribe,I Know Who You Are,"[Hook] Now, you ain't gotta lie to me, I know ...",04:17,joy,"hip-hop,hip hop",The Crusader,1st January 2003,B min,99,...,Scribe,Feel Good Extreme Randomizer Soundtrack 01,1.000000,Scribe,Cruisin In the Evening,1.000000,i know who you are,scribe,83131,1
1,JPEGMAFIA,Xanny Davis Jr.,"[Intro: JPEGMAFIA] Two xans, two xans Look at ...",02:47,sadness,"hip hop,cloud rap",SIN MIEDO,15th July 2024,C Maj,145,...,Ayesha Erotica,Real Messy Bitch,0.965441,Ayesha Erotica,Fins Sink or Swim Pt. 2,0.965086,xanny davis jr,jpegmafia,46547,1
2,$uicideboy$,$low Motion Potion,[Intro: ODDY NUFF DA SNOW LEOPARD] A SLMDNKASR...,02:06,anger,"hip hop,trap,cloud rap",Antarctica,26th January 2016,D Maj,106,...,$uicideboy$,Royal,1.000000,$uicideboy$,$TORAGE,1.000000,low motion potion,$uicideboy$,292,1
3,Dashboard Confessional,Dusk and Summer,"She smiled in a big way, the way a girl like t...",03:21,sadness,"alternative rock,emo",Dusk And Summer,1st January 2007,D# Maj,86,...,The Blue Stones,Black Holes Solid Ground,0.985417,"Dabin,Lexi Norton",In Flames,0.983517,dusk and summer,dashboard confessional,22251,0
4,Saba Lou,Good Habits and Bad,[Verse 1] Early to bed Early to rise Picking m...,02:00,joy,"rock,heavy metal,punk",Until the End,9th October 2012,F# Maj,128,...,Peggy Lee,Why Dont You Do Right?,0.988120,Peggy Lee,Why Dont You Do Right Get Me Some Money Too,0.988120,good habits and bad,saba lou,81674,0


df_song_test shape: (21625, 43)


,Artist(s),song,text,Length,emotion,Genre,Album,Release Date,Key,Tempo,...,Similar Artist 2,Similar Song 2,Similarity Score 2,Similar Artist 3,Similar Song 3,Similarity Score 3,song_normalized,artist_normalized,original_index,Explicit_binary
0,!!!,Even When the Waters Cold,Friends told her she was better off at the bot...,03:47,sadness,hip hop,Thr!!!er,29th April 2013,D min,105,...,Toby Keith,Drinks After Work,0.983719,Space,Neighbourhood,0.983236,even when the waters cold,!!!,0,0
1,!!!,One Girl / One Boy,"Well I heard it, playing soft From a drunken b...",04:03,sadness,hip hop,Thr!!!er,29th April 2013,A# min,117,...,When In Rome,Heaven Knows,0.990905,Justice Crew,Everybody,0.984483,one girl one boy,!!!,1,0
2,!zeesh,Camp Fire,[Verse 1: Bill] Yeah You don't got bars that d...,01:52,anger,hip hop,Campfire (Freestyle),25th April 2023,D Maj,101,...,Knucks,From Rome With Love,0.947579,"FILV,Edmofo,Emma Peters,Jvstin",All the Good Girls Go To Hell,0.940189,camp fire,!zeesh,2,1
3,"""Weird Al"" Yankovic",Amish Paradise,[Verse 1] As I walk through the valley where I...,03:22,joy,"rock,pop,comedy",Bad Hair Day,31st December 1996,G# Maj,81,...,"""Weird Al"" Yankovic",George Of The Jungle,0.999424,Danity Kane,Showstopper Remix,0.980017,amish paradise,"""weird al"" yankovic",3,0
4,"""Weird Al"" Yankovic",Word Crimes,[Intro] Everybody shut up! (Woo!) Everyone lis...,03:43,joy,"rock,pop,comedy",Mandatory Fun,15th July 2014,G Maj,122,...,Summer Thieves,Bottom of Your Face,0.989532,Summer Thieves,NFTs,0.989532,word crimes,"""weird al"" yankovic",4,0


df_original shape: (108125, 43)


In [50]:
# Zero-shot results dataframe
df_zero_shot_train = pd.read_csv(path_df_zero_train, nrows=NROWS)
df_zero_shot_test = pd.read_csv(path_df_zero_test, nrows=NROWS)
print("df_zero_shot_train shape:", df_zero_shot_train.shape)
print("df_zero_shot_test shape:", df_zero_shot_test.shape)
df_zero_shot_train['explicit_score'] = (df_zero_shot_train['explicit_score'] >= 0.5).astype(int)
df_zero_shot_test['explicit_score'] = (df_zero_shot_test['explicit_score'] >= 0.5).astype(int)

display(df_zero_shot_train.head())
display(df_zero_shot_test.head())


df_zero_shot_train shape: (86500, 2)
df_zero_shot_test shape: (21625, 2)


,original_index,explicit_score
0,31344,0
1,45251,0
2,13407,0
3,45259,1
4,36024,0


,original_index,explicit_score
0,83131,1
1,46547,1
2,292,1
3,22251,0
4,81674,1


In [39]:
# df prompting results
df_prompting_train = pd.read_csv(prompting_path_train, nrows=NROWS)
df_prompting_test = pd.read_csv(prompting_path_test, nrows=NROWS)

print("df_prompting_train shape:", df_prompting_train.shape)
print("df_prompting_test shape:", df_prompting_test.shape)
display(df_prompting_train.head())
display(df_prompting_test.head())

df_prompting_train shape: (86500, 2)
df_prompting_test shape: (21625, 2)


,original_index,explicit_score
0,31344,0
1,45251,0
2,13407,0
3,45259,1
4,36024,0


,original_index,explicit_score
0,83131,1
1,46547,1
2,292,1
3,22251,0
4,81674,0


In [ ]:
# 

In [45]:
# We are going to check the indexes to make sure everything aligns
if df_zero_shot_train['original_index'].equals(df_prompting_train['original_index']):
    # print("Indexes match between zero-shot and prompting results.")
    if df_song_train['original_index'].equals(df_prompting_train['original_index']):
        print('Correct')
    if df_song_test['original_index'].equals(df_song_test['original_index']):
        print('Correct')

    if df_zero_shot_test['original_index'].equals(df_prompting_test['original_index']):
        print("Indexes match between zero-shot and prompting results for test and train set as well.")
    else:
        print("Indexes do not match between zero-shot and prompting results for test set. Please check.")

else:
    print("Indexes do not match between zero-shot and prompting results. Please check.")



Correct
Correct
Indexes match between zero-shot and prompting results for test and train set as well.


In [52]:
new_labeles = []

# Start with train set
for i in range(len(df_song_train) +len(df_song_test)):
    # Remember df is completly splitted
    # So I have to find i in train or test

    if i in df_song_train['original_index'].values:
        # idx = df_song_train.loc[df_song_train['original_index'] == i].index
        idx = df_song_train.loc[df_song_train['original_index'] == i].index[0]
        df_label = df_song_train.iloc[idx]['Explicit_binary']
        zero_shot_label = df_zero_shot_train.iloc[idx]['explicit_score']
        prompting_label = df_prompting_train.iloc[idx]['explicit_score']
    else:
        # idx = df_song_test.loc[df_song_test['original_index'] == i].index
        idx = df_song_test.loc[df_song_test['original_index'] == i].index[0]

        df_label = df_song_test.iloc[idx]['Explicit_binary']
        zero_shot_label = df_zero_shot_test.iloc[idx]['explicit_score']
        prompting_label = df_prompting_test.iloc[idx]['explicit_score']

    # print(f'zero_shot_label:[{zero_shot_label}]')
    # print(f'prompting_label:[{prompting_label}]')
    # print(f'df_label:[{df_label}]')

    if zero_shot_label == prompting_label and zero_shot_label == df_label:
        new_labeles.append(df_label)
        # If all are equal, keep the original label
    elif zero_shot_label ==1 and prompting_label ==1:
        new_labeles.append(1)
        # If both models say explicit, label as explicit
    elif df_label == 1 and (zero_shot_label ==0 and prompting_label ==0):
        new_labeles.append(1)
        # If original is explicit and one model says explicit, label as explicit
    else:
        # Majority vote 
        votes = df_label + zero_shot_label + prompting_label
        if votes >=2:
            new_labeles.append(1)
        else:
            new_labeles.append(0)




# Save the final dataset
new_csv_path = f"{path_DATA}/spotify_dataset_pseudo_labeling.csv"

df_original['pseudo_label_explicit'] = new_labeles
df_original.to_csv(new_csv_path, index=False)



In [55]:
mismached_index = df_original[df_original['Explicit_binary'] != df_original['pseudo_label_explicit']]['original_index'].values
# display(df_original.head())

print("df_original shape:", df_original.shape)
print("Mismached indexes count:", len(mismached_index))

df_original shape: (108125, 44)
Mismached indexes count: 11481
